In [1]:
import pandas as pd
import numpy as np
import psycopg2
import json
import requests
import re
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
from selenium.webdriver.support.ui import WebDriverWait
import selenium.webdriver.support.expected_conditions as EC
from tqdm.notebook import tqdm
import time

In [2]:
class postgres_conn:
    def getConn(self):
        try:
            connection = psycopg2.connect(user='postgres',
                                          password='postgres_007',
                                          host="3.pgsql.db.1digitalstack.com",
                                          port='5432',
                                          database='postgres')

            cursor = connection.cursor()
            # Print PostgreSQL Connection properties
            print(connection.get_dsn_parameters(), "\n")

            # Print PostgreSQL version
            return cursor, connection

        except (Exception, psycopg2.Error) as error:
            print("Error while connecting to PostgreSQL", error)
            return error, error
    def getConn2(self):
        try:
            connection = psycopg2.connect(user='postgres',
                                          password='reporting_007',
                                          host="2.pgsql.db.1digitalstack.com",
                                          port='5432',
                                          database='postgres')

            cursor = connection.cursor()
            # Print PostgreSQL Connection properties
            print(connection.get_dsn_parameters(), "\n")

            # Print PostgreSQL version
            return cursor, connection

        except (Exception, psycopg2.Error) as error:
            print("Error while connecting to PostgreSQL", error)
            return error, error


    def close_connection(self, cursor, connection):
        # closing database connection.
        if(connection):
            cursor.close()
            connection.close()
            print("PostgreSQL connection is closed")
            
pg = postgres_conn()
conn = pg.getConn()
conn2 = pg.getConn2()

{'user': 'postgres', 'channel_binding': 'prefer', 'dbname': 'postgres', 'host': '3.pgsql.db.1digitalstack.com', 'port': '5432', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'sslsni': '1', 'ssl_min_protocol_version': 'TLSv1.2', 'gssencmode': 'disable', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

{'user': 'postgres', 'channel_binding': 'prefer', 'dbname': 'postgres', 'host': '2.pgsql.db.1digitalstack.com', 'port': '5432', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'sslsni': '1', 'ssl_min_protocol_version': 'TLSv1.2', 'gssencmode': 'disable', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 



In [ ]:
id= input("Enter the category id:\n")# 231515

print(' input date in format as shown here 2022-07-22')
startingdate=input('Starting Date')
endingdate= input('Ending Date')

In [ ]:
# cat data from production
query = f"""
select *
from ready.ready_product
where  crawl_date between '{startingdate}' and '{endingdate}' and category_id='{id}'
"""

ready_product = pd.read_sql_query(query, conn[1])

ready_product.dropna(subset=['asin'], inplace=True)

ready_product

In [ ]:
asin_cnt=ready_product.groupby(["pipeline_id"])["asin"].nunique().reset_index()
asin_cnt

In [ ]:
count=[100]
alist=list(asin_cnt[asin_cnt['asin'].isin(count)]['pipeline_id'])

ready_product=ready_product[ready_product['pipeline_id'].isin(alist)]
ready_product

In [ ]:
ask=input("enter anything you want. just wanted to confirm that upd are uploaded or not")

In [ ]:
upd_df=pd.read_excel(r"C:\Users\visha\Downloads\category_upd (1).xlsx")
upd_df

In [ ]:
ready_product_with_upd=pd.merge(ready_product,upd_df,left_on='rank',right_on='BSR 2', how='left')
ready_product_with_upd

In [ ]:
country_code=input('enter country code like "AU". \n')

In [ ]:
# brand mapping

query3= f"""
select *
from  entity.product_master
where  channel_sku_id in {tuple(ready_product['asin'])} and country_code='{country_code}'
"""

product_master = pd.read_sql_query(query3, conn[1])
product_master['brand_name']=product_master['brand_name'].str.lower()
product_master

In [ ]:
msi_df=pd.merge(ready_product_with_upd[['asin','unit_per_day', 'price','rank']],product_master[['channel_sku_id','brand_name','title']], left_on='asin',right_on='channel_sku_id', how='left')#.fillna('others')

msi_df

In [ ]:
msi_df.sort_values('brand_name', inplace= True)

brands_df=msi_df[['asin','brand_name']].drop_duplicates('asin')

msi_df.sort_values('price', inplace= True)

prices_df=msi_df[['asin','price']].drop_duplicates('asin')

msii_df=pd.merge(brands_df, prices_df, on='asin', how='outer')
msii_df

gh_df=msii_df[(msii_df['brand_name'].isna())|(msii_df['price'].isna())]
gh_df

In [ ]:

asinList = list(gh_df['asin'])
len(asinList)

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--incognito")
driver = webdriver.Chrome(executable_path = r"C:\Users\Tanuja Thakur\OneDrive\Desktop\chromedriver.exe",chrome_options = chrome_options)


In [ ]:
        l1 = "https:amazon.fr"
        print(l1)
        driver.get(l1)

In [ ]:
timeout = 10
locator = "imgTagWrapperId"
productDetails = {}
lastIdx = 0
for val in tqdm(range(0,len(asinList))):
    try:
   
        i = asinList[val]
        print(val,i)
        productDetails[i] = {}
        lastIdx = val
        link = l1+"/dp/" + i
        print(link)
        driver.get(link)
        WebDriverWait(driver, timeout).until(EC.visibility_of_element_located((By.ID, locator)))
        try:
            TitleVal = driver.find_element(By.ID,'title')
            productDetails[i]['Title'] = TitleVal.text
            print(TitleVal.text)
        
        except:
            productDetails[i]['Title'] = "Not Found"
            print("Product Title not found")


        try:
            BrandVal = driver.find_element(By.ID,'bylineInfo')
            productDetails[i]['Brand_Subscript'] = BrandVal.text
            print(BrandVal.text)

        except:
            productDetails[i]['Brand_Subscript'] = "Not Found"
            print("Brand from subscript not found")
        
        try:
            soup = BeautifulSoup(driver.page_source, "html.parser")
            z=soup.find_all('span', class_='a-offscreen')
            
            price=z[0].text
            productDetails[i]['Price'] =price

            print(price)
        except:
            productDetails[i]['Price'] = "Not Found"
            print("Price not found")
            
         
            
            
    except:
        pass

In [ ]:
df_productDetails = pd.DataFrame.from_dict(productDetails,orient='index')
df_productDetails['Brand_Subscript'] = df_productDetails['Brand_Subscript'].str.replace('^Brand:\s','').str.replace('\sStore','').str.replace('^Visit\sthe\s','').str.replace('^Marca:\s','').str.replace('^Visita\slo\sdi\s','').str.replace('^Visita\sla\sde\s','').str.replace('^Merk:\s','').str.replace('^De\s',''). str.replace('openen','').str.replace('^Marque :\s','').str.replace('^Visiter la boutique\s','').str.lower()
df_productDetails['Price']=df_productDetails['Price'].str.replace("€","")
my_df= pd.merge(msii_df, df_productDetails, left_on='asin', right_on=df_productDetails.index, how= "left")
my_df

In [ ]:
my_df.brand_name.fillna(my_df.Brand_Subscript, inplace=True)
my_df.price.fillna(my_df.Price, inplace=True)

my_df

In [ ]:
ms_df=pd.merge(ready_product_with_upd[['asin','rank','unit_per_day','title']],my_df[['asin','brand_name','price']], on="asin", how='left')
ms_df

In [ ]:
top_brand=ms_df.groupby(['brand_name']).unit_per_day.sum().reset_index(name='upd').sort_values(by=['upd'],ascending=False)

top_brand

# Brand's number of Asin in BSR 100


In [ ]:
brand_wise_asin_in_bsr_top100=ms_df.drop_duplicates('asin').groupby(['brand_name']).count().reset_index() #count() #.sort_values(by=['count'],ascending=False)

brand_wise_asin_in_bsr_top100

# brands wise count of asin in top 10 BSR

In [ ]:
brand_wise_asin_in_bsr_top10=ms_df[ms_df['rank']<=10].drop_duplicates('asin').groupby(['brand_name']).count().reset_index() #count() #.sort_values(by=['count'],ascending=False)

brand_wise_asin_in_bsr_top10

# Brand wise average price

In [ ]:
client_id=input('client id ')#'130'

In [ ]:
tablename2='client_{0}.serp_data_{1}'.format(client_id,client_id)

In [ ]:
feature=input('Please insert Feature from Keyword_category:')
feature

In [ ]:
feature=tuple(['239','240','241'])

In [ ]:
query5=f"""
select*
from entity.keyword_category
where feature = '{feature}'

"""

keyword_df = pd.read_sql_query(query5, conn[1])
keyword_df

In [ ]:
keyword_df=keyword_df[keyword_df['sub_category']=='ear care']
keyword_df

In [ ]:
keystone_key1= "'" +  ("'" + "','".join(name.replace("'", r"''") for name in list(keyword_df['keyword'])) + "'") + "'"
keystone_key1

In [ ]:
#keyword_list=pd.read_excel('keystone_keyword.xlsx')


In [ ]:
query4 = f"""
select *
from  {tablename2} 
where  crawl_date between '{startingdate}' and '{endingdate}' and channel_id= '{1}' and keyword  in ({str(keystone_key1)[1:-1]})
"""

serp_data = pd.read_sql_query(query4, conn2[1])

In [ ]:
serp_data

In [ ]:
serp_data['brand_name']=serp_data['brand_name'].str.lower()

avg_price=serp_data.groupby(['channel_sku_id','brand_name'])['price'].mean().reset_index()

avg_price1=avg_price.groupby(['brand_name'])['price'].mean().reset_index()

brand_wise_avg_price=avg_price1[avg_price1['brand_name'].isin(list(top_brand['brand_name']))]

brand_wise_avg_price

# Total review count

In [ ]:


total_review_count=serp_data.groupby(['channel_sku_id','brand_name'])['rating_count'].max().reset_index()

total_review_count1=total_review_count.groupby(['brand_name'])['rating_count'].sum().reset_index()

brand_wise_total_review_count=total_review_count1[total_review_count1['brand_name'].isin(list(top_brand['brand_name']))]

brand_wise_total_review_count

# Average rating

In [ ]:
avg_rating=serp_data.groupby(['channel_sku_id','brand_name'])['rating'].mean().reset_index()

avg_rating1=avg_rating.groupby(['brand_name'])['rating'].mean().reset_index()

brand_wise_avg_rating=avg_rating1[avg_rating1['brand_name'].isin(list(top_brand['brand_name']))]

brand_wise_avg_rating

# AVERAGE REVIEW VELOCITY

In [ ]:
ratings_df=serp_data[['channel_sku_id','sku_title','brand_name','rating','rating_count','crawl_date']]
max_count=ratings_df.groupby(['channel_sku_id','brand_name'])['rating_count'].max().reset_index()
min_count=ratings_df.groupby(['channel_sku_id','brand_name'])['rating_count'].min().reset_index()

dif_review=pd.merge(min_count,max_count, on=['channel_sku_id','brand_name'], how= 'outer')
dif_review=dif_review.fillna(0)
dif_review['change_in_rating']=dif_review['rating_count_y']-dif_review['rating_count_x']

brand_wise_velocity=dif_review[dif_review['brand_name'].isin(list(top_brand['brand_name']))]

brand_wise_review_velocity=brand_wise_velocity.groupby(['brand_name'])['change_in_rating'].sum().reset_index()

brand_wise_review_velocity

# GENERIC KEYWORD COVERAGE

In [ ]:
generic_serp_data=serp_data[serp_data['sponsored']!=1]

generic_kwd_coverage=generic_serp_data.drop_duplicates(subset=['keyword','brand_name'])[['brand_name','keyword']]

generic_kwd_coverage1=generic_kwd_coverage.groupby(['brand_name']).count().reset_index()

generic_kwd_coverage1['kwd_coverage_%']=generic_kwd_coverage1['keyword']/len(list(keyword_df['keyword']))

brand_wise_generic_kwd_coverage=generic_kwd_coverage1[generic_kwd_coverage1['brand_name'].isin(list(top_brand['brand_name']))]

brand_wise_generic_kwd_coverage

# PAID KEYWORD COVERAGE

In [ ]:
paid_serp_data=serp_data[serp_data['sponsored']==1]

paid_kwd_coverage=paid_serp_data.drop_duplicates(subset=['keyword','brand_name'])[['brand_name','keyword']]

paid_kwd_coverage1=paid_kwd_coverage.groupby(['brand_name']).count().reset_index()

paid_kwd_coverage1['kwd_coverage_%']=paid_kwd_coverage1['keyword']/len(list(keyword_df['keyword']))

brand_wise_paid_kwd_coverage=paid_kwd_coverage1[paid_kwd_coverage1['brand_name'].isin(list(top_brand['brand_name']))]

brand_wise_paid_kwd_coverage

In [ ]:
one=pd.merge(top_brand,brand_wise_asin_in_bsr_top100[['brand_name','asin']],on='brand_name',how='left').rename(columns={'asin':'No. of ASINs in BSR (Top 100)'})

In [ ]:

two=one.merge(brand_wise_asin_in_bsr_top10[['brand_name','asin']],on='brand_name', how='left').rename(columns={'asin':'No. of ASIN in BSR (Top 10)'})

In [ ]:
three=two.merge(brand_wise_avg_price[['brand_name','price']],on='brand_name',how='left').rename(columns={'price':'AVR Price'})

In [ ]:
three=three.rename(columns={'upd':'Est. Volume'})

In [ ]:
four=three.merge(brand_wise_total_review_count[['brand_name','rating_count']],on='brand_name',how='left')

In [ ]:
five=four.merge(brand_wise_avg_rating[['brand_name','rating']],on='brand_name',how='left')

In [ ]:
six=five.merge(brand_wise_review_velocity[['brand_name','change_in_rating']],on='brand_name',how='left')

In [ ]:
seven=six.merge(brand_wise_generic_kwd_coverage[['brand_name','kwd_coverage_%']],on='brand_name', how='left').rename(columns={'kwd_coverage_%':'Keyword Coverage (Organic)'})

In [ ]:
eight=seven.merge(brand_wise_paid_kwd_coverage[['brand_name','kwd_coverage_%']],on='brand_name', how='left').rename(columns={'kwd_coverage_%':'Keyword Coverage (Paid)','rating_count':'Total Review Count','rating':'Average Rating','change_in_rating':'Average Review Velocity','brand_name':'Brand'})

In [ ]:
eight['Total Number of ASINs']=""
eight['Market Share (volume)']=""
eight['Estimated Revenue']=""
eight['Market Share']=""
eight['Country']="France"
eight['Category']=""
eight['Sub-category']=""
eight['Sub-Sub Category']=""
eight['Marketplace']="Amazon"
eight['Tier']=""
eight['Ranking']=""

In [ ]:
eight=eight[['Country','Category','Sub-category','Sub-Sub Category','Marketplace','Tier','Ranking',
             'Brand','Total Number of ASINs','No. of ASINs in BSR (Top 100)','No. of ASIN in BSR (Top 10)',
      'AVR Price','Est. Volume','Market Share (volume)','Estimated Revenue',
             'Market Share','Keyword Coverage (Organic)','Keyword Coverage (Paid)','Total Review Count','Average Rating','Average Review Velocity']]
eight

In [ ]:
fileout="551 category analysis.xlsx" 

In [ ]:
with pd.ExcelWriter(fileout, engine='xlsxwriter')as writer:
        eight.to_excel(writer,sheet_name='analysis' ,startrow=0,startcol=0,index=False)
        ms_df.to_excel(writer,sheet_name='cat_data' ,startrow=0,startcol=0,index=False)
#         serp_data.to_excel(writer,sheet_name='serp_data' ,startrow=0,startcol=0,index=False)

In [ ]:
conn[0].close()
conn[1].close()
conn2[0].close()
conn2[1].close()

In [ ]:
msi_df.sort_values('brand_name', inplace= True)

brands_df=msi_df[['asin','brand_name']].drop_duplicates('asin')

msi_df.sort_values('price', inplace= True)

prices_df=msi_df[['asin','price']].drop_duplicates('asin')

msii_df=pd.merge(brands_df, prices_df, on='asin', how='outer')
msii_df

gh_df=msii_df[(msii_df['brand_name'].isna())|(msii_df['price'].isna())]
gh_df

In [ ]:
unique_sku=msi_df[['asin']].drop_duplicates()
unique_sku

In [ ]:
unique_sku['brand']=unique_sku.apply(lambda x: msi_df[msi_df['asin']==x]['brand_name'][0] , axis=1)
unique_sku

In [ ]:
unique_sku['price']=unique_sku.apply(lambda x: )

In [ ]:
msi_df[msi_df['asin']=='B073TW58NZ']

In [ ]:
msi_df['brand_name'][0]

In [ ]:
pip install yake

In [ ]:
import yake


In [ ]:
cvdf=pd.read_csv(r"C:\Users\nitin\Downloads\traditional laptop.csv")
mygh=list(cvdf['Product Details'])
mygh


In [ ]:
text="".join(mygh)
text

In [ ]:

kw_extractor = yake.KeywordExtractor()
#text ="""HP 15s, 11th Gen Intel Core i3, 8GB RAM/512GB SSD 15.6-inch(39.6 cm) Micro-Edge Anti-Glare FHD Laptop/Alexa Built-in/Win 11/Intel UHD Graphics/Dual Speakers/ MS Office 2021/1.69 Kg, 15s-fr2511TU Lenovo IdeaPad Slim 3 Intel Celeron N4020 4th Gen 15.6" (39.62cm) HD Thin & Light Laptop (8GB/256GB SSD/Windows 11/Office 2021/2Yr Warranty/3months Game Pass/Platinum Grey/1.7Kg), 81WQ00MQIN """ 
language = "es"
max_ngram_size = 3
deduplication_threshold = 0.5
numOfKeywords = 20
custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_threshold, top=numOfKeywords, features=None)
keywords = custom_kw_extractor.extract_keywords(text)
for kw in keywords:
    print(kw)

In [ ]:
len(keywords)


In [ ]:
import streamlit as st
import yake
import pandas as pd
import numpy as np
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from spacy import displacy
import re

In [ ]:
st.write('<style>div.row-widget.stRadio > div{flex-direction:row;}</style>', unsafe_allow_html=True)
HTML_WRAPPER = """<div style="overflow-x: hidden; border: 1px solid #e6e9ef; border-radius: 0.25rem; padding: 1rem; margin-bottom: 2.5rem">{}</div>"""


st.sidebar.title("Yet Another Keyword Extractor (YAKE)")
st.sidebar.markdown("""
Unsupervised Approach for Automatic Keyword Extraction using Text Features.
https://liaad.github.io/yake/
"""
)

st.sidebar.markdown("""
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/LIAAD/yake/blob/gh-pages/notebooks/YAKE_tutorial.ipynb)
""")

st.sidebar.header('Parameters')

#side bar parameters
max_ngram_size = st.sidebar.slider("Select max ngram size", 1, 10, 3)
deduplication_thresold = st.sidebar.slider("Select deduplication threshold", 0.5, 1.0, 0.9)
numOfKeywords = st.sidebar.slider("Select number of keywords to return", 1, 50, 10)
deduplication_algo = st.sidebar.selectbox('deduplication function', ('leve','jaro','seqm'),2)



input_text_demo = [
    "Extracting keywords from texts has become a challenge for individuals and organizations  as the information grows in complexity and size . The need to automate this task so that text can be processed in a timely and adequate manner has led to the emergence of automatic keyword extraction tools . Yake is a novel  feature-based system for multi-lingual keyword extraction , which supports texts of different sizes, domain or languages. Unlike other approaches, Yake does not rely on dictionaries nor thesauri, neither is trained against any corpora. Instead, it follows an unsupervised approach which builds upon features extracted from the text, making it thus applicable to documents written in different languages without the need for further knowledge. This can be beneficial for a large number of tasks and a plethora of situations where access to training corpora is either limited or restricted.",
    "Google is acquiring data science community Kaggle. Sources tell us that Google is acquiring Kaggle, a platform that hosts data science and machine learning   competitions. Details about the transaction remain somewhat vague , but given that Google is hosting   its Cloud Next conference in San Francisco this week, the official announcement could come as early   as tomorrow.  Reached by phone, Kaggle co-founder CEO Anthony Goldbloom declined to deny that the   acquisition is happening. Google itself declined 'to comment on rumors'.   Kaggle, which has about half a million data scientists on its platform, was founded by Goldbloom   and Ben Hamner in 2010. The service got an early start and even though it has a few competitors   like DrivenData, TopCoder and HackerRank, it has managed to stay well ahead of them by focusing on its   specific niche. The service is basically the de facto home for running data science  and machine learning   competitions.  With Kaggle, Google is buying one of the largest and most active communities for   data scientists - and with that, it will get increased mindshare in this community, too   (though it already has plenty of that thanks to Tensorflow and other projects).   Kaggle has a bit of a history with Google, too, but that's pretty recent. Earlier this month,   Google and Kaggle teamed up to host a $100,000 machine learning competition around classifying   YouTube videos. That competition had some deep integrations with the Google Cloud Platform, too.   Our understanding is that Google will keep the service running - likely under its current name.   While the acquisition is probably more about Kaggle's community than technology, Kaggle did build   some interesting tools for hosting its competition and 'kernels', too. On Kaggle, kernels are   basically the source code for analyzing data sets and developers can share this code on the   platform (the company previously called them 'scripts').  Like similar competition-centric sites,   Kaggle also runs a job board, too. It's unclear what Google will do with that part of the service.   According to Crunchbase, Kaggle raised $12.5 million (though PitchBook says it's $12.75) since its   launch in 2010. Investors in Kaggle include Index Ventures, SV Angel, Max Levchin, Naval Ravikant,   Google chief economist Hal Varian, Khosla Ventures and Yuri Milner",
    "Genius quietly laid off a bunch of its engineers — now can it survive as a media company?.  Genius, which raised $56.9 million on the promise that it would one day annotate the entire internet, has been losing its minds. In January, the company quietly laid off a quarter of its staff, with the bulk of the cuts coming from the engineering department. In a post on the Genius blog at the time, co-founder Tom Lehman told employees that Genius planned to shift its emphasis away from the annotation platform that once attracted top-tier investors in favor of becoming a more video-focused media company.  'After taking a careful look at the company and our priorities,' Lehman wrote, 'we’ve had to make some tough decisions about how we want to spend our resources. And unfortunately this meant that today we laid off some very talented people.' The company then took the unusual step of posting the Genius usernames of those it had laid off — 12 full-time and five part-time employees.  'WE NEEDED TO SHIFT RESOURCES FROM CAPTURING KNOWLEDGE... TOWARD PACKAGING AND DISTRIBUTING KNOWLEDGE.' At the same time, Lehman noted that the company was continuing to hire for roles in video and sales. The company recently redesigned its homepage with expanded space for editorial content and advertising. It has also been deepening its Behind the Lyrics partnership with Spotify, for which it contributes a mix of song lyrics and factoids that pop up in a slideshow format when you’re listening to popular songs.  'The change we made in January was in recognition of the fact that we needed to shift resources from capturing knowledge — which we've been doing almost exclusively for the past five years — toward packaging and distributing knowledge into easy-to-consume formats like video and Spotify Behind the Lyrics,' Lehman told The Verge.  It’s not unusual for tech companies to transform over time, though typically they are loath to lay off engineers. But Genius’ shift is more dramatic than most: going from all-encompassing annotator of the internet to a more traditional media company model, chasing video views alongside an ever-growing number of well-capitalized competitors. The move illustrates the company’s difficulty attracting contributors — and an audience for their contributions — particularly outside of the music world.  In an interview last week, Lehman said the company had turned to video in an effort to reach its core audience‚ which continues to be rap fans, beyond its website and mobile apps. 'Video makes it a little bit more accessible,' he said. 'I love the Genius website. One of my favorite websites. But it can be a little frustrating to use. You have to be really, really dedicated to learn everything about a song on Genius. You've got to be down to click and read a lot.'  Genius’ videos to date have included interviews with artists about their craft, and a series where rappers perform original freestyle verses. One of Lehman’s favorite videos traces a whistle sample featured in a series of popular songs back to its origins in a Quentin Tarantino movie. Last week, Genius posted a video about rapper Lil Yachty learning how to make pizza. The company is also investing in original editorial content, aggregating news headlines, doing Q&As with artists like the Track Burnaz, and writing short profiles. ",
]



windowSize = 1

#User text in content
st.header('Demo')

selected_input_text = st.selectbox("Select sample text", input_text_demo)
text = st.text_area("Selected text", selected_input_text, 330)
language = "english"

#use yake to extract keywords
custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_thresold, dedupFunc=deduplication_algo, windowsSize=windowSize, top=numOfKeywords, features=None)
keywords = custom_kw_extractor.extract_keywords(text)




#get keywords and their position
ents = []
text_lower = text.lower()

keywords_list = str(keywords[0][0])
for m in re.finditer(keywords_list, text_lower): 
    d = dict(start = m.start(), end = m.end(), label = "")
    ents.append(d)
    
for i in range(1, len(keywords)):
    kwords = str(keywords[i][0])
    keywords_list += (', ' + kwords)
    for m in re.finditer(kwords, text_lower): 
        d = dict(start = m.start(), end = m.end(), label = "")
        ents.append(d)
        
#sort the result by ents, as ent rule suggests
sort_ents = sorted(ents, key=lambda x: x["start"])

st.header('Output')

result_view = st.radio("Choose visualization type",('Highlighting', 'Word cloud', 'Table'), index=0)
if result_view == 'Highlighting':
    #use spacy to higlight the keywords
    ex = [{"text": text,
        "ents": sort_ents,
        "title": None}]

    html = displacy.render(ex, style="ent", manual=True)
    #html = html.replace("\n", " ")
    st.write(HTML_WRAPPER.format(html), unsafe_allow_html=True)
elif result_view == "Table":
    #tabular data (columns: keywords, score)
    df = pd.DataFrame(keywords, columns=("keywords","score"))
    st.table(df)
    
else:
    #create and generate a word cloud image
    wordcloud = WordCloud(width = 1000, height = 600, max_font_size = 80, 
                min_font_size=10, prefer_horizontal=1, 
                max_words=numOfKeywords,
                background_color="white", 
                collocations=False, 
                regexp = r"\w[\w ']+").generate(keywords_list)

    #display the generated image
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    st.pyplot(plt)

In [ ]:
keywords

In [ ]:
import numpy as np

In [ ]:
size_x=5
size_y=3

In [ ]:
matrix = np.zeros ((size_x, size_y))
matrix

In [ ]:
        for x in range(size_x):
            matrix [x, 0] = x
        for y in range(size_y):
            matrix [0, y] = y

In [ ]:
matrix

In [ ]:
        for x in range(size_x):
            matrix [x, 0] = x
            print(matrix)

In [ ]:
candidate_string='I am a good boy. My name is NITIN Bhayana'

In [ ]:
from segtok.segmenter import split_multi
from segtok.tokenizer import web_tokenizer, split_contractions

In [ ]:
sentences_str = [w for w in split_contractions(web_tokenizer(candidate_string.lower())) if not (w.startswith("'") and len(w) > 1) and len(w) > 0]

In [ ]:
sentences_str

In [ ]:
    def getTag( word, i):
        try:
            w2 = word.replace(",","")
            float(w2)
            return "d"
        except:
            cdigit = len([c for c in word if c.isdigit()])
            calpha = len([c for c in word if c.isalpha()])
            if ( cdigit > 0 and calpha > 0 ) or (cdigit == 0 and calpha == 0) or len([c for c in word if c in .exclude]) > 1:
                return "u"
            if len(word) == len([c for c in word if c.isupper()]):
                return "a"
            if len([c for c in word if c.isupper()]) == 1 and len(word) > 1 and word[0].isupper() and i > 0:
                return "n"
        return "p"

In [ ]:
z=enumerate('mylist',2)

In [ ]:
print(list(z))